# docfiller

:::{eval-rst}
.. currentmodule:: module_utilities.docfiller
:::

It is common to want to share documentation information across functions/methods.  The module {mod}`~module_utilities.docfiller` provides a way to do this:

For example, if you have these function:

In [1]:
def a_function(x, y, z):
    """
    A function.

    A longer description


    Parameters
    ----------
    x : float
        An x param
    y : int
        A y param
    z : str
        A z param


    Returns
    -------
    output : float
        An output
    """


def b_function(x, y, z):
    """
    A different function.

    A longer description of b_function


    Parameters
    ----------
    x : float
        An x param
    y : float
        A different y param
    z : str
        A z param


    Returns
    -------
    output : float
        An different output
    """

It would be nice to share info.  {class}`DocFiller` provides a few ways to do this. 

First, we can define common docstrings like so:

In [2]:
from module_utilities.docfiller import DocFiller

In [3]:
docstrings = """
Parameters
----------
x : float
    An x param
z : str
    A z param
"""

d = DocFiller.from_docstring(docstrings, combine_keys="parameters")

d.data

{'summary': '',
 'extended_summary': '',
 'parameters': {'x': 'x : float\n    An x param',
  'z': 'z : str\n    A z param'},
 'returns': '',
 'yields': '',
 'notes': '',
 'warnings': '',
 'other_parameters': '',
 'attributes': '',
 'methods': '',
 'references': '',
 'examples': '',
 'x': 'x : float\n    An x param',
 'z': 'z : str\n    A z param'}

This parses the `docstring` (using numpydoc).  Then, we can define functions as:

In [4]:
from textwrap import dedent


@d()
def a_function2(x, y, z):
    """
    A function.

    A longer description


    Parameters
    ----------
    {x}
    y : int
        A y param
    {z}


    Returns
    -------
    output : float
        An output
    """


@d()
def b_function2(x, y, z):
    """
    A different function.

    A longer description of b_function


    Parameters
    ----------
    {x}
    y : float
        A different y param
    {z}


    Returns
    -------
    output : float
        An different output
    """


assert dedent(a_function.__doc__) == a_function2.__doc__
assert dedent(b_function.__doc__) == b_function2.__doc__

print(a_function2.__doc__)
print(b_function2.__doc__)


A function.

A longer description

Parameters
----------
x : float
    An x param
y : int
    A y param
z : str
    A z param

Returns
-------
output : float
    An output


A different function.

A longer description of b_function

Parameters
----------
x : float
    An x param
y : float
    A different y param
z : str
    A z param

Returns
-------
output : float
    An different output



## Using templates

Better still is to generalize the template:

In [5]:
@d.update(
    summary="A function.",
    extended_summary="""
    A longer description
    """,
    y="""
    y : int
        A y param
    """,
    output="""
    output : float
        An output
    """,
).dedent().decorate
def a_function3(x, y, z):
    """
    {summary}

    {extended_summary}


    Parameters
    ----------
    {x}
    {y}
    {z}


    Returns
    -------
    {output}
    """


assert a_function3.__doc__ == a_function2.__doc__


@d.update(
    summary="A different function.",
    extended_summary="A longer description of b_function",
    y="""
    y : float
        A different y param
    """,
    output="""
    output : float
        An different output
    """,
).dedent()(a_function3)
def b_function3(x, y, z):
    pass


assert b_function3.__doc__ == b_function2.__doc__

Note that for `b_function3`, we used the template/docstring from `a_function3`.  {class}`docfiller.DocFiller` defaults to using
the functions docstring for replacement.  But you can pass a string or function to use as the template to be filled.

## General shared docs

Another approach is to define shared parameters, with slight variations in there own Docfiller instance, and use this
for replacement.

In [6]:
da = DocFiller.from_docstring(
    """
    A function.

    A longer description

    Parameters
    ----------
    y : int
        A y param

    Returns
    -------
    output : float
        An output
    """,
    # this makes parameters and returns available at top level.
    # otherwise, have to access using `parameters.y`, etc.
    combine_keys=["parameters", "returns"],
)

db = DocFiller.from_docstring(
    """
    A different function.

    A longer description of b_function

    Parameters
    ----------
    y : float
        A different y param

    Returns
    -------
    output : float
        An different output
    """,
    combine_keys=["parameters", "returns"],
)

In [7]:
@d.append(da)(a_function3)
def a_function4():
    pass


assert a_function4.__doc__ == a_function2.__doc__


@d.append(db)(a_function3)
def b_function4():
    pass


assert b_function4.__doc__ == b_function4.__doc__

## Define parameters/returns that have different key values

You may find that you can define a host of shared parameter documentation, except that there are a few edge cases.
For example, say we have a parameter `beta` that in some cases is an array, and in others is a float.  You could handle this as follows

In [8]:
docstring = """
Parameters
----------
beta_float | beta : float
    A float value for beta
beta_array | beta : array-like
    An array of beta values

x : float
    x parameter
y : float
    y parameter
"""

d = DocFiller.from_docstring(docstring, combine_keys="parameters")

# using `.decorate` uses the decorator without template or parameters


@d.decorate
def func_float(x, y, beta):
    """
    A thing


    Parameters
    ----------
    {x}
    {y}
    {beta_float}
    """


@d.decorate
def func_array(x, y, beta):
    """
    An array thing


    Parameters
    ----------
    {x}
    {y}
    {beta_array}
    """


print(func_float.__doc__)
print(func_array.__doc__)


A thing

Parameters
----------
x : float
    x parameter
y : float
    y parameter
beta : float
    A float value for beta


An array thing

Parameters
----------
x : float
    x parameter
y : float
    y parameter
beta : array-like
    An array of beta values



Or, if everyting else is the same, you could do:

In [9]:
# back to call because passing parameters
@d(beta=d["beta_float"], summary="A thing")
def func_float2(x, y, beta):
    """
    {summary}


    Parameters
    ----------
    {x}
    {y}
    {beta}
    """


@d(func_float2, beta=d["beta_array"], summary="An array thing")
def func_array2(x, y, beta):
    pass


assert func_float2.__doc__ == func_float.__doc__
assert func_array2.__doc__ == func_array.__doc__

Better still, you can use {meth}`docfiller.Docfiller.assign_keys`:

In [11]:
@d.assign_keys(beta="beta_float")(summary="A thing")
def func_float3(x, y, beta):
    """
    {summary}


    Parameters
    ----------
    {x}
    {y}
    {beta}
    """


@d.assign_keys(beta="beta_array")(func_float3, summary="An array thing")
def func_array3(x, y, beta):
    pass


assert func_float3.__doc__ == func_float.__doc__
assert func_array3.__doc__ == func_array.__doc__

In [17]:
from module_utilities.docfiller import DocFiller

d = DocFiller.from_docstring(
    """
    Parameters
    ----------
    x : int
        x param
    y : float
        y param
    z0 | z : int
        z int param
    z1 | z : float
        z float parma
    """,
    combine_keys="parameters",
)


@d()
def func0():
    """
    Parameters
    ----------
    {x}
    {y}
    {z0}
    """


print(func0.__doc__)


Parameters
----------
x : int
    x param
y : float
    y param
z : int
    z int param

